[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bamirandab/nlp-disaster-tweets/blob/main/notebooks/model.ipynb)

In [2]:
from google.colab import files
files.upload() #upload kaggle.json

Saving clean.py to clean.py


{'clean.py': b'import re\r\nimport nltk\r\nnltk.download(\'punkt\')\r\nfrom nltk.corpus import stopwords\r\nfrom nltk.tokenize import word_tokenize\r\nimport mapply\r\n\r\n\r\nmapply.init(\r\n    n_workers=-1,\r\n    chunk_size=100,\r\n    max_chunks_per_worker=8,\r\n    progressbar=False\r\n)\r\n\r\nnltk.download(\'stopwords\')\r\nstop_words = stopwords.words()\r\n\r\ndef search_keywords(x, tweet):\r\n  result = re.search(\'[{letter}]\\w+\'.format(letter=x),tweet)\r\n  if result: \r\n    return True\r\n  else:\r\n    return False\r\n\r\ndef clean_tweet(tweet):\r\n  tweet_tokens = re.sub(\'\\s+\', \' \', tweet.lower()).split(\' \')\r\n  tweet_clean = \' \'.join([word for word in tweet_tokens if not word in stop_words])\r\n  return tweet_clean\r\n\r\n\r\ndef drop_word(x, tweet):\r\n  return re.sub(\'[{letter}]\\w+\'.format(letter=x),\'\',tweet)\r\n\r\ndef drop_url(tweet):\r\n  return re.sub(r\'http\\S+\',\'\',tweet)\r\n\r\n\r\ndef clean_df(x):\r\n    x_clean_temp = x["text"].mapply(lamb

In [7]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [8]:
!kaggle competitions download -c nlp-getting-started -p ../data

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 80.0MB/s]


In [4]:
!pip install mapply

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76 kB 3.7 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 115 kB 17.1 MB/s 


In [9]:
from clean import clean_df
from zipfile import ZipFile
import pandas as pd
import os


In [10]:
path = os.path.join('../data/nlp-getting-started.zip')

with ZipFile(path, 'r') as zipObj:
#    # Extract all the contents of zip file in current directory
   zipObj.extractall('../data')

In [11]:
data_path = '../data/'
train_df = pd.read_csv(data_path + 'train.csv', encoding='utf-8')
test_df = pd.read_csv(data_path + 'test.csv', encoding='utf-8')

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [14]:
train_labels = train_df["target"].to_list()
train_sentences = clean_df(train_df)
test_sentences = clean_df(test_df)

In [16]:
# Number of examples to use for training
training_size = 20000

# Vocabulary size of the tokenizer
vocab_size = 10000

# Maximum length of the padded sequences
max_length = 32

# Output dimensions of the Embedding layer
embedding_dim = 16

In [17]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Parameters for padding and OOV tokens
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

# Initialize the Tokenizer class
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)

# Generate the word index dictionary
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# Generate and pad the training sequences
training_sequences = tokenizer.texts_to_sequences(train_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Generate and pad the testing sequences
testing_sequences = tokenizer.texts_to_sequences(test_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Convert the labels lists into numpy arrays
training_labels = np.array(train_labels)

In [19]:
import tensorflow as tf
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 16)            160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 24)                408       
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [23]:
num_epochs = 30

# Train the model
history = model.fit(training_padded, training_labels, epochs=num_epochs, verbose=2)

Epoch 1/30
238/238 - 2s - loss: 0.6709 - accuracy: 0.5782 - 2s/epoch - 8ms/step
Epoch 2/30
238/238 - 1s - loss: 0.5565 - accuracy: 0.7502 - 852ms/epoch - 4ms/step
Epoch 3/30
238/238 - 1s - loss: 0.4033 - accuracy: 0.8418 - 808ms/epoch - 3ms/step
Epoch 4/30
238/238 - 1s - loss: 0.3259 - accuracy: 0.8698 - 867ms/epoch - 4ms/step
Epoch 5/30
238/238 - 1s - loss: 0.2782 - accuracy: 0.8901 - 833ms/epoch - 4ms/step
Epoch 6/30
238/238 - 1s - loss: 0.2432 - accuracy: 0.9062 - 808ms/epoch - 3ms/step
Epoch 7/30
238/238 - 1s - loss: 0.2159 - accuracy: 0.9178 - 817ms/epoch - 3ms/step
Epoch 8/30
238/238 - 1s - loss: 0.1921 - accuracy: 0.9264 - 814ms/epoch - 3ms/step
Epoch 9/30
238/238 - 1s - loss: 0.1728 - accuracy: 0.9366 - 776ms/epoch - 3ms/step
Epoch 10/30
238/238 - 1s - loss: 0.1568 - accuracy: 0.9421 - 765ms/epoch - 3ms/step
Epoch 11/30
238/238 - 1s - loss: 0.1418 - accuracy: 0.9475 - 803ms/epoch - 3ms/step
Epoch 12/30
238/238 - 1s - loss: 0.1305 - accuracy: 0.9514 - 819ms/epoch - 3ms/step
Epoc

In [32]:
testing_labels = model.predict(testing_padded)

In [33]:
test_labels = [1 if label > 0.5 else 0 for label in testing_labels ]

In [38]:
test_df['target'] = pd.Series(test_labels)
test_df[['id','target']].to_csv("test.csv", index=False)